In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense
import numpy as np
import cv2
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

In [2]:
dataset_path = '/content/drive/MyDrive/animal'
width = height = 224

idg = ImageDataGenerator(

    rescale = 1./255,
    horizontal_flip = True,
    #zoom_range = 0.1,
    brightness_range = (0.9,1.1),
    #rotation_range = 15,
    validation_split = 0.2 #validation data
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    subset = "training"
)

val_data = idg.flow_from_directory(  #eamal taghirat bar roye tasavir
    dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    subset = "validation"
)

Found 1042 images belonging to 5 classes.
Found 257 images belonging to 5 classes.


In [4]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights = 'imagenet',
    input_shape=(width,height,3),
    pooling = 'avg',
)

model = tf.keras.models.Sequential([
        base_model,
        #Dropout(0.3),
        # Flatten(),
        # Dense(1024,activation='relu'),
        Dense(256,activation='relu'),
        Dense(5,activation='softmax')

])

In [5]:
for layer in base_model.layers[:-2]:  #freeze layer
  layer.trainable = False

In [6]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [7]:
model.fit(train_data,validation_data=val_data,epochs=10)

Epoch 1/10
33/33 [==============================] - 476s 14s/step - loss: 0.3384 - accuracy: 0.8925 - val_loss: 0.1040 - val_accuracy: 0.9767
Epoch 2/10
33/33 [==============================] - 65s 2s/step - loss: 0.0811 - accuracy: 0.9712 - val_loss: 0.0712 - val_accuracy: 0.9728
Epoch 3/10
33/33 [==============================] - 65s 2s/step - loss: 0.0250 - accuracy: 0.9904 - val_loss: 0.0704 - val_accuracy: 0.9728
Epoch 4/10
33/33 [==============================] - 66s 2s/step - loss: 0.0164 - accuracy: 0.9962 - val_loss: 0.0661 - val_accuracy: 0.9767
Epoch 5/10
33/33 [==============================] - 66s 2s/step - loss: 0.0069 - accuracy: 0.9990 - val_loss: 0.0641 - val_accuracy: 0.9767
Epoch 6/10
33/33 [==============================] - 57s 2s/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0603 - val_accuracy: 0.9805
Epoch 7/10
33/33 [==============================] - 65s 2s/step - loss: 0.0028 - accuracy: 0.9990 - val_loss: 0.0699 - val_accuracy: 0.9767
Epoch 8/10
33/33 [

In [9]:
model.save('animals_recognition.h5')
